In [ ]:
# @title 설치

# 형태소 분석기 Mecab 설치
!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

# 라이브러리 설치
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import re
from PIL import Image
from io import BytesIO
from nltk.tokenize import RegexpTokenizer
import nltk
from gensim.models import Word2Vec
from gensim.models import doc2vec
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Mecab
from konlpy.tag import Okt
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 28.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.6/581.6 kB 4.1 MB/s eta 0:00:00
  Created wheel for mecab-python: filename=mecab_python-1.0.0-py3-none-any.whl size=1229 sha256=b27ea001bd331ad6a4d34762b753c326d81aa2c18087e66c6095e1124cbec960
  Stored in directory: /root/.cache/pip/wheels/ec/41/c8/ce8fb469be2547ce596c4000613806505dc54c296aae45da71
Successfully built mecab-python
Install mecab-ko
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 1381k  100 1381k    0     0   441k      0  0:00:03  0:00:03 --:--:--  997k
mecab-0.996-ko-0.9.2/
mecab-0.996-ko-0.9.2/example/
mecab-0.996-ko-0.9.2/example/ex

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# @title 데이터 정리

# 데이터 불러오기

## read_csv 안 작성내용은 코랩 기준으로 작성된거에요.
## pc 버전에서 작업하신다면,
## 작업 위치에 넣어두고 csv파일을 넣어두고, read_csv('한국관광공사_열린관광지목록_20220804.csv',encoding = 'cp949') 작성하시면 될 거에요!
data1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/deepdive/콘텐츠 기반 추천시스템/한국관광공사_열린관광지목록_20220804.csv',encoding = 'cp949')
data2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/deepdive/콘텐츠 기반 추천시스템/대구광역시_관광코스 정보_20210906.csv',encoding = 'cp949')
data3 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/deepdive/콘텐츠 기반 추천시스템/한국문화관광연구원_관광자원정보_20230510.csv',encoding = 'cp949')
data4 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/deepdive/콘텐츠 기반 추천시스템/전라남도 광양시_추천여행코스.csv',encoding = 'UTF8')
data5 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/deepdive/콘텐츠 기반 추천시스템/울산광역시 남구_울남9경_20220831.csv',encoding = 'cp949')
data6 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/deepdive/콘텐츠 기반 추천시스템/경상북도 칠곡군_관광명소 현황_2016년 6월..csv',encoding = 'cp949')
data7 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/deepdive/콘텐츠 기반 추천시스템/경상남도 밀양시_관광지정보_20221024.csv', encoding='cp949')
data8 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/deepdive/콘텐츠 기반 추천시스템/충청남도_테마여행.csv', encoding='cp949')
data9 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/deepdive/콘텐츠 기반 추천시스템/부산광역시 동구_이바구길 관광코스_20220921.csv', encoding='cp949')

# 각 데이터 프레임에서 필요한 칼럼만 가져오기
df1 = data1[['관광지명','설명']]
df2 = data2[['관광지', '코스설명2']]
df3 = data3[['자원명', '자원설명']]
df4 = data4[['body__rows__arNm', 'body__rows__arCt']]
df5 = data5[['울남 9경', '소개글']]
df6 = data6[['관광명소명', '소개']]
df7 = data7[['관광지명', '관광지소개']]
df8 = data8[['이름', '상세내용']]
df9 = data9[['관광지명', '내용']]

# 칼럼명 통일시켜주기(df1은 동일하니, df2,3만 진행)
df2 = df2.rename(columns = {'관광지' : '관광지명', '코스설명2' : '설명'})
df3 = df3.rename(columns = {'자원명' : '관광지명', '자원설명' : '설명'})
df4 = df4.rename(columns = {'body__rows__arNm' : '관광지명', 'body__rows__arCt' : '설명'})
df5 = df5.rename(columns = {'울남 9경' : '관광지명', '소개글' : '설명'})
df6 = df6.rename(columns = {'관광명소명' : '관광지명', '소개' : '설명'})
df7 = df7.rename(columns = {'관광지명' : '관광지명', '관광지소개' : '설명'})
df8 = df8.rename(columns = {'이름' : '관광지명', '상세내용' : '설명'})
df9 = df9.rename(columns = {'관광지명' : '관광지명', '내용' : '설명'})

# 데이터프레임 하나로 합치기
df = pd.concat([df1,df2, df3,df4,df5,df6, df7, df8, df9], ignore_index=True)

# 생성된 데이터프레임 확인
df

,관광지명,설명
0,인천 개항장역사문화공간,1883년 인천 개항장 126년 세월의 흔적이 쌓인 개항기 역사와 문화의 만남의 장...
1,인천 월미문화의거리,"인천의 대표적인 관광지 중의 하나인 월미도에는 월미도 문화의 거리, 월미공원, 월미..."
2,인천 연안부두해양광장,서해 도서로 운항하는 여객선의 출발지이다. 싱싱한 생선과 젓갈 등을 저렴한 가격으로...
3,인천 하나개해수욕장,하나개해수욕장은 샘꾸미 선착장에서 차로 10분이면 도착할 수 있다. 무의도에서 가장...
4,양평 세미원,"세미원 연꽃박물관은 연꽃이라는 단일한 테마 아래 연꽃관련 생활 용품, 고서, 음식 ..."
...,...,...
831,증산공원,"1982년 개장한 증산공원은 약 20,000m의 면적에 지역 주민들을 위한 운동 시..."
832,초량전통시장 야시장,"다양한 먹거리와 신나는 공연, 오감만족의 체험으로 동구의 밤을 밝히다 1910년대 ..."
833,스카이웨이 전망대,"보행자의 안전과 원활한 차량 통행, 주차 공간 확보 등 지역 주민과 관광객들의 편의..."
834,유치환의 우체통,경남여고 교장을 역임하고 좌천동에서 불의의 교통사고로 타계한 청마 유치환 선생을 기...


In [ ]:
df[450:500]

,index,관광지명,설명
450,505,청운리안씨고가,경상남도 문화재 자료 제호 청운 리 안씨 고가 조선 후기 경상도 상류층 주거 양식 ...
451,506,초동연가길,초동 연가 낙동강 변 자연 습지 반월 습지 밀양시 성장동력 사업 조성 봄 꽃 양귀비...
452,507,밀양추원재,경상남도 문화재 자료 제호 조선 전기 문신 이자 학자 김숙자 선생 터 잡고 아들 김...
453,508,추화산성,밀양시 해발 주 화산 산마루 돌 성 밀양읍 성 으로부터 동쪽 약 지점 신라 가야 낙...
454,509,밀양충혼탑,동족상잔 비극 조국 수호 위해 장렬 히 산화 지역 출신 육 해 공군 경찰 노무 애국...
455,510,칠탄서원,단장면 촌리 칠 서원 임진왜란 석 동산 의병 충의 선조 판관 영 천 군수 오한 손 ...
456,511,밀양파크골프클럽,파크 골프장 시설 관리 공단 관리 경남 최대 규모 파크 골프장 개장 코스 부터 코스...
457,512,평리산대추마을,평리 마을 대량 대추 생산 마을 대추 생산 훼손 그대로 자연 주변 관광지 여가 활동...
458,513,하남명품십리길,하남읍 낙동강 변 노을 명품 십리 길 구절초 백일홍 조성 명품 십리 거리 정도 백일...
459,514,명례강변공원,밀양아리랑 오토 캠핑장 주변 명례 강변 만여 평의 메밀 꽃 단지 조성 봄 가을 메밀...


In [ ]:
# @title 전처리

# 데이터 전처리
df = df.dropna(how = 'any') # Null 값이 존재하는 행 제거
df['설명'] = df['설명'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글 외 문자 제거

# 불용어 제거 - 직접
stopwords = ['인천','울산','광양','칠곡','대구', '밀양', '충청', '부산','년','월','일','수','인','하나','개요','이라는','의','가','이','은','들','는','좀','잘','과',
             '도','를','으로','자','에','와','한','하다','로','을','에서','위','에는','고','되다','이며','다','그','그리고','장소이다','것','있다','처럼',
             '곳','때','중','이름','모습','남쪽','두','등','않다','많다','하고','까지','적','이다', '이고', '이나', '로서']
okt = Okt()
tokenized_data = []

for sentence in df['설명']:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    tokenized_data.append(temp_X)

# 불용어 제거 - 동사, 형용사
for i, list in enumerate(tokenized_data):
  text = ' '.join(list)

  okt = Okt()
  words = okt.pos(text, norm=True, stem=True)  # 형태소 분석 및 품사 태깅

  # 동사 불용어 제거를 위한 품사 태그 (동사: Verb, 형용사: Adjective)
  stop_tags = ['Verb', 'Adjective']

  # 불용어가 아닌 단어들만 필터링하여 리스트에 저장
  filtered_words = [word for word, pos in words if pos not in stop_tags]

  tokenized_data[i] = filtered_words

for i,item in enumerate(tokenized_data):
  tokenized_data[i] = ' '.join(s for s in item)

df['설명'] = tokenized_data

df = df.reset_index()

df

<ipython-input-11-09d3b283390d>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['설명'] = df['설명'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글 외 문자 제거
<ipython-input-11-09d3b283390d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['설명'] = df['설명'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글 외 문자 제거
<ipython-input-11-09d3b283390d>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['설명'] = tokenized_data


,관광지명,설명
0,인천 개항장역사문화공간,개항 장 세월 흔적 개항 기 역사 문화 만남 장소 옛 영사관 중구청 일본 제 은행 ...
1,인천 월미문화의거리,대표 관광지 월미도 월미도 문화 거리 공원 파크 월미도 문화 거리 보행자 전용 도로...
2,인천 연안부두해양광장,서해 도서 운항 여객선 출발 지다 생선 젓갈 가격 구입 종합 어시장 수협 직영 회 ...
3,인천 하나개해수욕장,개 해수욕장 샘 선착장 차로 도착 무의도 가장 갯벌 뜻 개 해수욕장 길이 해변 밀가...
4,양평 세미원,세미 원 연꽃 박물관 연꽃 단일 테마 아래 연꽃 관련 생활 용품 고서 음식 유물 전...
...,...,...
831,증산공원,개장 증산 공원 약 면적 지역 주민 운동 시설 산책로 중앙 광장 조성 증산 전망대 ...
832,초량전통시장 야시장,먹거리 공연 오감 만족 체험 동구 밤 년대 형성 시작 년대 초량 전통 시장 내 건 ...
833,스카이웨이 전망대,보행자 안전 차량 통행 주차 공간 확보 지역 주민 관광객 편의 위해 산복도로 망양로...
834,유치환의 우체통,경남 여고 교장 역임 좌천동 불의 교통사고 타계 청마 유치환 선생 기 리기 위해 지...


In [ ]:
# @title 문서화 : : df에서 '설명' 열에 있는 한국어 텍스트 데이터를 형태소 분석하고, 해당 텍스트가 어떤 '관광지명'과 연관되는지를 태깅
# 태깅? 텍스트나 문장의 각 요소에 레이블이나 태그를 할당하는 작업, 본 코드에서는 표를 리스트화한다고 생각할 수 있음

mecab = Mecab()

tagged_corpus_list = []

for index, row in tqdm(df.iterrows(), total=len(df)):
  text = row['설명']
  tag = row['관광지명']
  tagged_corpus_list.append(TaggedDocument(tags=[tag], words=[s for s in mecab.morphs(text) if len(s) >= 2 ]))

print('문서의 수 :', len(tagged_corpus_list))

tagged_corpus_list[0]

100%|██████████| 779/779 [00:00<00:00, 849.74it/s]

문서의 수 : 779


TaggedDocument(words=['개항', '세월', '흔적', '개항', '역사', '문화', '만남', '장소', '영사관', '구청', '일본', '은행', '지점', '일본', '은행', '지점', '일본', '은행', '지점', '근대', '건축', '그대로', '장소', '우선', '주식회사', '아트', '플렛', '제물포', '구락부', '역사', '흔적', '직접', '문화', '관광', '해설', '배치', '예약', '해설', '해설', '도보', '관광', '가족', '단위', '여행', '단체', '호응'], tags=['인천 개항장역사문화공간'])

In [ ]:
# @title doc2vec 모델 생성 및 train

model = doc2vec.Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, workers=8, window=8) # 여기 하이퍼파라미터 튜닝

# Vocabulary 빌드
model.build_vocab(tagged_corpus_list)

# Doc2Vec 학습
model.train(tagged_corpus_list, total_examples = model.corpus_count, epochs = 50) # 에포크 늘려보기

# 모델 저장
model.save('dart.doc2vec')

In [ ]:
# @title doc2vec 모델 테스트 -  평가 지표 : 상위 3개 유사도 평균
place = input('어느 곳과 유사한 관광지를 찾으시나요? :')
similar_doc = model.docvecs.most_similar(place)
rounded_similar_doc = pd.DataFrame([(doc, round(similarity, 2)) for doc, similarity in similar_doc])

# 상위 3개 유사도의 평균 구하기
top_3_similarities = [similarity for doc, similarity in similar_doc[:3]] # k=3
average_similarity = sum(top_3_similarities) / len(top_3_similarities)
print(f"상위 3개 관광지의 유사도 평균: {average_similarity:.2f}")

rounded_similar_doc

어느 곳과 유사한 관광지를 찾으시나요? :금오동천
상위 3개 관광지의 유사도 평균: 0.60


<ipython-input-42-9f63ffd69f8d>:3: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  similar_doc = model.docvecs.most_similar(place)


,0,1
0,강천산 군립공원,0.62
1,월악산국립공원(단양군),0.60
2,월악산국립공원 단양분소(도락산),0.57
3,월악산국립공원 단양분소(금수산),0.56
4,월악산국립공원 단양분소(제비봉),0.56
5,월악산,0.56
6,성불계곡,0.55
7,예산 봉수산 자연휴양림&수목원,0.52
8,비슬산 참꽃군락지,0.52
9,칠갑산도립공원,0.52


In [ ]:
# @title Word2vec 모델 생성 및 train
sentences = [tagged_doc.words for tagged_doc in tagged_corpus_list]
model2 = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=5, workers=4, sg=0)

print(model2.wv.most_similar("등산"))
print(model2.wv.most_similar("개항"))
print(model2.wv.most_similar("휴양"))

[('코스', 0.9995440244674683), ('시간', 0.9994964003562927), ('가을', 0.9993902444839478), ('부터', 0.9993708729743958), ('가족', 0.9993610382080078), ('오르', 0.9993502497673035), ('에서', 0.9993346929550171), ('시작', 0.9993335008621216), ('또한', 0.9993274807929993), ('마다', 0.9993157386779785)]
[('일제', 0.9957808256149292), ('북쪽', 0.9957612752914429), ('간직', 0.9956862330436707), ('철새', 0.9956411123275757), ('공주', 0.9956391453742981), ('다리', 0.995627224445343), ('물결', 0.9955742359161377), ('송림', 0.9955250024795532), ('매우', 0.9955236911773682), ('가을', 0.9955234527587891)]
[('시설', 0.9992817044258118), ('함께', 0.9992783665657043), ('주변', 0.999267041683197), ('에서', 0.9992408752441406), ('위치', 0.9992274045944214), ('관광', 0.9992233514785767), ('전국', 0.99920654296875), ('모두', 0.999203622341156), ('산책', 0.9991912841796875), ('자리', 0.9991856813430786)]


In [ ]:
# @title Word2vec 모델에서 유사도가 높은 단어들을 기반으로 관광지 추천하기
from collections import Counter

keywords = ["등산", "개항", "휴양"]
similar_word_lists = {}

# 각 키워드별로 유사한 단어 찾기
for keyword in keywords:
    similar_words = model2.wv.most_similar(keyword, topn=3) # 유사한 단어 3개만 사용, 예를 들어 "등산"에서는 "코스", "시간", "정상"만 사용
    similar_word_lists[keyword] = similar_words

# 유사한 단어들을 많이 포함하는 관광지명 찾기
matched_tags_counter = Counter()
matched_tags_details = {}

for keyword in keywords:
    print(f"유사한 단어들({keyword}):")
    for word, similarity in similar_word_lists[keyword]:
        matched_tags = []
        for tagged_doc in tagged_corpus_list:
            if word in tagged_doc.words:
                matched_tags.extend(tagged_doc.tags)
        matched_tags_counter.update(matched_tags)
        matched_tags_details[word] = matched_tags

print("가장 많이 등장한 관광지명 Top 5:")
for tag, count in matched_tags_counter.most_common(5):
    print(f"관광지명: {tag} - 등장 횟수: {count}")

유사한 단어들(등산):
유사한 단어들(개항):
유사한 단어들(휴양):
가장 많이 등장한 관광지명 Top 5:
관광지명: 까꼬막 카페 - 등장 횟수: 6
관광지명: 남원관광지 - 등장 횟수: 6
관광지명: 한라산국립공원(서귀포시) - 등장 횟수: 5
관광지명: 봉수산자연휴양림 - 등장 횟수: 5
관광지명: 옥룡사지동백림 - 등장 횟수: 5
